In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2
import json

In [3]:
# load data from json

with open('Data_to_Otomoto.json') as file:
    Data_to_Otomoto = json.loads(file.read())

insert_table = Data_to_Otomoto["insert_table"]
car_brand = Data_to_Otomoto["car_brand"]
table_cars = Data_to_Otomoto["table_cars"]
CREATE_TABLE = Data_to_Otomoto["CREATE_TABLE"]

In [4]:
eur = 4.57
print(eur)


4.57


In [5]:
def get_price(soup):
    Cena = soup.find('span',attrs={"class":"offer-price__number"}).text
    Cena = Cena.replace(" ","").replace(",",".")
    if "PLN" in Cena:
        Cena = Cena.replace("PLN","")
        Cena = float(Cena)
    elif "EUR" in Cena:
        Cena = Cena.replace("EUR","")
        Cena = float(Cena)
        Cena = Cena * eur
    else:
        Cena = np.NaN
    return Cena

In [6]:
def take_id(soup):
    ID = soup.find('div',attrs={"class":"offer-content__metabar"})
    ID = ID.find('div')
    ID = ID.find('span',attrs={"id":"ad_id"}).text
    return ID

In [7]:
def take_details(soup):
    details = {}    
    for info in soup.find_all('li',attrs={"class":"offer-params__item"}):
        key = info.find('span').text.strip().replace(" ","_").replace("(","").replace(")","")
        key = key.replace("ś","s").replace("ż","z").replace("ó","o").replace("ć","c").replace("ę","e")
        key = key.replace("ł","l").replace("ą","a")
        if key in insert_table:
            value = info.find('div').text.strip()
            details[key] = value
    return details

In [8]:
def take_all(soup):
    try:
        dict_all = {}
        dict_all["ID"] = take_id(soup)
        dict_all["Cena"] = get_price(soup)
        details = take_details(soup)
        return dict_all | details
    except:
        pass

In [9]:
def data_clean(details):
    try:
        details['Przebieg'] = details['Przebieg'][:-2].replace(" ","")
    except:
        pass
    try:
        details['Moc'] = details['Moc'][:-2].replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Pojemnosc_skokowa'] = details['Pojemnosc_skokowa'][:-3].replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_W_Miescie'] = details['Spalanie_W_Miescie'][:-7].replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_W_Cyklu_Mieszanym'] = details['Spalanie_W_Cyklu_Mieszanym'][:-7].replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_Poza_Miastem'] = details['Spalanie_Poza_Miastem'][:-7].replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Emisja_CO2'] = details['Emisja_CO2'][:-4].replace(" ","").replace(",",".")
    except:
        pass
    return details

In [10]:
type_list = ['Cena','Moc','Pojemnosc_skokowa','Liczba_drzwi','Przebieg','Rok_produkcji',
             'Spalanie_W_Miescie','Emisja_CO2','Liczba_miejsc','Spalanie_W_Cyklu_Mieszanym',
             'Spalanie_Poza_Miastem']
#     'ID'

In [11]:
# def data_clean_type(dict_all):


# for row in table_cars.index:
#     for col in type_list:
#         cel = table_cars.at[row,col]
#         if type(cel) != float:
#             table_cars.at[row,col] = np.nan
            

In [12]:
# table_cars.at[0,'ID']

In [13]:
 '''(ID float NOT NULL,
        Cena float,
        Moc float,
        Pojemnosc_skokowa float,
        Liczba_drzwi float,
        Przebieg float,
        Rok_produkcji float,
        Spalanie_W_Miescie float,
        Emisja_CO2 float,
        Liczba_miejsc float,
        Spalanie_W_Cyklu_Mieszanym float,
        Spalanie_Poza_Miastem float,'''


'(ID float NOT NULL,\n       Cena float,\n       Moc float,\n       Pojemnosc_skokowa float,\n       Liczba_drzwi float,\n       Przebieg float,\n       Rok_produkcji float,\n       Spalanie_W_Miescie float,\n       Emisja_CO2 float,\n       Liczba_miejsc float,\n       Spalanie_W_Cyklu_Mieszanym float,\n       Spalanie_Poza_Miastem float,'

In [14]:
# car_brand = ["abarth",
#         "alfa-romeo",
#         "aston-martin",
#         "audi",
#         "bentley",
#         "bmw",
#         "bugatti",
#         "cadillac",
#         "chevrolet",
#         "chrysler",
#         "citroën",
#         "dacia",
#         "daewoo",
#         "daihatsu",
#         "dodge",
#         "donkervoort",
#         "ds",
#         "ferrari",
#         "fiat",
#         "fisker",
#         "ford",
#         "honda",
#         "hummer",
#         "hyundai",
#         "infiniti",
#         "iveco",
#         "jaguar",
#         "jeep",
#         "kia",
#         "ktm",
#         "lada",
#         "lamborghini",
#         "lancia",
#         "land-rover",
#         "landwind",
#         "lexus",
#         "lotus",
#         "maserati",
#         "maybach",
#         "mazda",
#         "mclaren",
#         "mercedes-benz",
#         "mg",
#         "mini",
#         "mitsubishi",
#         "morgan",
#         "nissan",
#         "opel",
#         "peugeot",
#         "porsche",
#         "renault",
#         "rolls-royce",
#         "rover",
#         "saab",
#         "seat",
#         "skoda",
#         "smart",
#         "ssangYong",
#         "subaru",
#         "suzuki",
#         "tesla",
#         "toyota",
#         "volkswagen",
#         "volvo"]
            

In [15]:
table_cars = pd.DataFrame(table_cars)

In [16]:
Link_ERROR = 0
Read_announcements = 0
new_row = {}
all_data = []
for brand in car_brand:
    brand_link = f"https://www.otomoto.pl/osobowe/{brand}/?search%5Border%5D=created_at%3Adesc&page="
    for page_main in range(1,2): 
        try:
            html_data_link = requests.get(brand_link + str(page_main))
            if brand in html_data_link.url:       
                soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
                links = []
                for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
                    if "https://www.otomoto.pl" in offer['href']:
                        links.append(offer['href']) 
                for page in links:
                    print(page)
                    html_data = requests.get(page)
                    soup = BeautifulSoup(html_data.content, 'html5lib')
                    dict_all = take_all(soup)
                    dict_all = data_clean(dict_all)
                    dict_all["Link"] = page
                    if dict_all:
                        all_data.append(dict_all)
                    Read_announcements += 1
                    clear_output(wait=True)
                    print("Read_announcements ", Read_announcements) 
        except Exception:
            print("Link_ERROR")
            continue
table_cars = table_cars.append(all_data, ignore_index=True)
#table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
delete_duplicates = Read_announcements - len(table_cars.index) 
print("Delete duplicates: ", delete_duplicates)
print("Finish")

Read_announcements  1591
Delete duplicates:  0
Finish


In [17]:
for row in table_cars.index:
    for col in type_list:
        cel = table_cars.at[row,col]
        try:
            cel = float(cel)
        except:
            table_cars.at[row,col] = np.nan
        

In [18]:
test = table_cars[['Pojemnosc_skokowa','Spalanie_W_Miescie','Spalanie_W_Cyklu_Mieszanym','Spalanie_Poza_Miastem','Emisja_CO2']]
test.head(50)

,Pojemnosc_skokowa,Spalanie_W_Miescie,Spalanie_W_Cyklu_Mieszanym,Spalanie_Poza_Miastem,Emisja_CO2
0,1368,NaN,NaN,NaN,NaN
1,1368,NaN,NaN,NaN,NaN
2,1368,NaN,NaN,NaN,NaN
3,1368,NaN,NaN,NaN,NaN
4,1368,NaN,NaN,NaN,NaN
5,1368,NaN,NaN,NaN,NaN
6,1400,NaN,NaN,NaN,NaN
7,1400,NaN,NaN,NaN,155
8,1368,NaN,NaN,NaN,139
9,1368,9,NaN,NaN,NaN


In [19]:
# #GIT
# Link_ERROR = 0
# Read_announcements = 0
# new_row = {}
# all_data = []
# for page_main in range(1,4):  
#     try:
#         html_data_link = requests.get(f"https://www.otomoto.pl/osobowe/?search%5Border%5D=created_at%3Adesc&page={page_main}")
#         soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
#     except Exception:
#         print("Link_ERROR")
#         continue
#     links = []
#     for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
#         if "https://www.otomoto.pl" in offer['href']:
#             links.append(offer['href']) 
#     for page in links:
#         html_data = requests.get(page)
#         soup = BeautifulSoup(html_data.content, 'html5lib')
#         dict_all = take_all(soup)
#         dict_all = data_clean(dict_all)
#         all_data.append(dict_all)
#         Read_announcements += 1
#         clear_output(wait=True)
#         print("Read_announcements ", Read_announcements)
# table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
# delete_duplicates = Read_announcements - len(table_cars.index) 
# print("Delete duplicates: ", delete_duplicates)
# print("Finish")

In [20]:
table_name = 'Otomoto_15'

In [21]:
table_cars.to_csv (table_name +'.csv', index = False, header=True)

In [22]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otomoto', user='barto', password='biznes')

In [23]:
cur = psql.cursor()

In [24]:
cur.execute('DROP TABLE IF EXISTS '+ table_name)
createTableCommand = 'CREATE TABLE ' + table_name + CREATE_TABLE 
cur.execute(createTableCommand)
psql.commit()

In [25]:
a = 0
for row in table_cars.itertuples(index=False):
    full=""
    full_fit="INSERT INTO " + table_name + "("
    for (column, insert) in zip(row, insert_table):
        if str(column) =='nan':
            continue
        column = str(column)
        column = column.replace("'", "''")
        full +="'"+column+"'"+", "
        full_fit += insert + ', '  
        
    full=full[:-2]
    full_fit=full_fit[:-2]
    full_fit=full_fit+')'
    Value_data='VALUES ('+full+') ON CONFLICT DO NOTHING;'
    insertDataCommand = full_fit+"\r\n"+Value_data
    
    cur.execute(insertDataCommand)
psql.commit()
print("Finish")

Finish
